In [2]:
import pickle
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
INTERSECTION = 0
BRIDGE = 1

In [50]:
#print G.nodes()
#print G.edges()
for (u,v) in G.edges()[:100]:
    print u,v,G.get_edge_data(u, v)

3480 b959 {'distance': 0.12749975, 'dailyvolume': 57431, 't_a': 28, 'lanes': 4, 'bridges': None, 'capacity_0': 8000, 'flow': 0, 'distance_0': 0.12749975, 'capacity': 8000, 't_0': 28}
3480 3455 {'distance': 0.5, 'dailyvolume': 2539, 't_a': 45, 'lanes': 1, 'bridges': [], 'capacity_0': 1400, 'flow': 0, 'distance_0': 0.5, 'capacity': 1400, 't_0': 45}
10611 10415 {'distance': 0.589999, 'dailyvolume': 5539, 't_a': 32, 'lanes': 1, 'bridges': [], 'capacity_0': 2000, 'flow': 0, 'distance_0': 0.589999, 'capacity': 2000, 't_0': 32}
10611 2364 {'distance': 0.0, 'dailyvolume': 2232, 't_a': 0, 'lanes': 7, 'bridges': [], 'capacity_0': 0, 'flow': 0, 'distance_0': 0.0, 'capacity': 0, 't_0': 0}
b415 9396 {'distance': 0.4499995, 'dailyvolume': 16329, 'capacity_0': 1950, 'lanes': 1, 'bridges': None, 't_a': 53, 'flow': 0, 'distance_0': 0.4499995, 'capacity': 1950, 't_0': 53}
b415 5261 {'distance': 0.4499995, 'dailyvolume': 67132, 'capacity_0': 5850, 'lanes': 3, 'bridges': None, 't_a': 53, 'flow': 0, 'dista

In [71]:
# Load road network
G = nx.read_gpickle("quick_traffic_model/input/graphMTC_CentroidsLength3int.gpickle")

# Make intersections node type 0 to separate from bridges
for n in G.nodes():
    G.node[n]['nodetype'] = INTERSECTION # intersection, not bridge

print 'finished loading road network'
G = G.to_undirected()

# Open bridge database
bridges_dict = None
with open('quick_traffic_model/input/20140114_master_bridge_dict.pkl','rb') as f:
    bridges_dict = pickle.load(f)

# Add list of bridges on each road segment
for b_num in bridges_dict.keys():
    bridge_edges = bridges_dict[b_num]['a_b_pairs_direct']
    for [u,v] in bridge_edges:
        G.edge[str(u)][str(v)]['bridges'].append('b' + b_num)
        
intersections_G = G.copy()

# For each bridge, make a new node connecting to the proper intersections
# in series, needs to be changed to in parallel for modularity to work properly
for (u,v) in G.edges():
    if len(G.edge[u][v]['bridges']) > 0:
        end1 = u
        end2 = v
        bridge_list = G.edge[u][v]['bridges']
        for bridge in bridge_list:
            data = G.get_edge_data(end1, end2)
            data['bridges'] = None
            data['distance'] = data['distance']/2
            data['distance_0'] = data['distance_0']/2
            data['t_0'] = data['t_0']/2
            G.remove_edge(end1,end2)
            G.add_node(bridge, nodetype=BRIDGE)
            G.add_edge(end1, bridge, attr_dict = data)
            G.add_edge(bridge, end2, attr_dict = data)
            end1 = u
            end2 = bridge

print 'finished adding bridges'
bridges_and_intersections_G = G.copy()

# Remove the intersections
for n in G.nodes():
    if G.node[n]['nodetype'] == INTERSECTION:
        adj_nodes = G.neighbors(n)
        for v1 in range(len(adj_nodes)):
            for v2 in range(v1+1, len(adj_nodes)):
                v1n = adj_nodes[v1]
                v2n = adj_nodes[v2]
                if G.node[v1n]['nodetype'] == BRIDGE and G.node[v2n]['nodetype'] == BRIDGE:
                    G.add_edge(v1n, v2n)
        G.remove_node(n)

print 'finished removing road intersections'
bridges_G = G.copy()

finished loading road network
finished adding bridges
finished removing road intersections


In [72]:
print(bridges_G.edges())

[('b828', 'b818'), ('b828', 'b819'), ('b828', 'b821'), ('b828', 'b1098'), ('b829', 'b827'), ('b829', 'b1141'), ('b824', 'b824'), ('b825', 'b792'), ('b825', 'b834'), ('b825', 'b806'), ('b826', 'b826'), ('b826', 'b673'), ('b827', 'b822'), ('b820', 'b1163'), ('b820', 'b864'), ('b820', 'b821'), ('b822', 'b843'), ('b823', 'b778'), ('b823', 'b823'), ('b910', 'b910'), ('b911', 'b837'), ('b911', 'b847'), ('b912', 'b884'), ('b912', 'b892'), ('b913', 'b914'), ('b913', 'b775'), ('b914', 'b929'), ('b914', 'b1009'), ('b915', 'b928'), ('b915', 'b893'), ('b916', 'b575'), ('b916', 'b908'), ('b916', 'b916'), ('b917', 'b895'), ('b917', 'b917'), ('b918', 'b925'), ('b918', 'b924'), ('b1097', 'b1096'), ('b1097', 'b1103'), ('b1097', 'b1201'), ('b1097', 'b1102'), ('b1097', 'b1131'), ('b1096', 'b1131'), ('b1096', 'b1103'), ('b1096', 'b1094'), ('b1096', 'b1201'), ('b1095', 'b537'), ('b1094', 'b1103'), ('b1094', 'b1100'), ('b1092', 'b1092'), ('b1090', 'b864'), ('b1090', 'b833'), ('b1090', 'b807'), ('b1090', 'b8